# Semantic Search - CES-WP-24-16

You can input a query or a question. The script then uses semantic search
to find relevant passages  from a Census.gov report:

https://www.census.gov/hfp/btos/downloads/CES-WP-24-16.pdf 

**PDF Title**:
Tracking Firm Use of AI in Real Time: A Snapshot from the Business Trends and
Outlook Survey



As model, we use: nq-distilbert-base-v1

It was trained on the Natural Questions dataset, a dataset with real questions from Google Search together with annotated data from 
Wikipedia providing the answer.



In [ ]:
#!pip install -U sentence-transformers

In [2]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import time
import gzip
import os
import torch
import pypdf

if torch.cuda.is_available():
    print("Found and using cuda/gpu...")
else:
    print("Warning: No GPU found. Please add GPU to your notebook")

Found and using cuda/gpu...


In [3]:
# We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
model_name = 'nq-distilbert-base-v1'
bi_encoder = SentenceTransformer(model_name)
top_k = 5  # Number of passages we want to retrieve with the bi-encoder

In [4]:
# As dataset, we use Simple English Wikipedia. Compared to the full English wikipedia, it has only
import requests
pdf_name = "CES-WP-24-16.pdf"
if os.path.exists(pdf_name):
    print(f"Found pdf ({pdf_name}) available locally, no need to (re)download it")
else:
    print(f"Coud NOT find pdf ({pdf_name}) available locally, attempting to download it...")
    url = f'https://www.census.gov/hfp/btos/downloads/{pdf_name}'
    response = requests.get(url)
    with open(pdf_name, 'wb') as f:
        f.write(response.content)
    print(f"Wrote file(?): {pdf_name}")

Found pdf (CES-WP-24-16.pdf) available locally, no need to (re)download it


In [8]:

# Open the PDF file
with open(pdf_name, 'rb') as file:
    reader = pypdf.PdfReader(file)
    text = ""
    # page_count = reader.page_count
    page_count = 0
    for page in reader.pages:
        page_count = page_count + 1
        text += page.extract_text() + "\n"

print(f"Extracted text from ({page_count}) pages, text size:({len(text)}")

Extracted text from (63) pages, text size:(120720


Now that we have extracted each page of text, and added them together, let's split that back apart...

In [ ]:
# To speed things up, pre-computed embeddings are downloaded.
# The provided file encoded the passages with the model 'nq-distilbert-base-v1'
if model_name == 'nq-distilbert-base-v1':
    embeddings_filepath = f'{pdf_name}-nq-distilbert-base-v1.pt'
    if os.path.exists(embeddings_filepath):
        print(f"Found existing (pre)embeddings file: {embeddings_filepath}")
    else:
        print(f"NO (pre)embeddings found, download/cache file: {embeddings_filepath} now...")
        util.http_get('http://sbert.net/datasets/simplewiki-2020-11-01-nq-distilbert-base-v1.pt', embeddings_filepath)

    corpus_embeddings = torch.load(embeddings_filepath)
    corpus_embeddings = corpus_embeddings.float()  # Convert embedding file to float
    if torch.cuda.is_available():
        corpus_embeddings = corpus_embeddings.to('cuda')
else:  # Here, we compute the corpus_embeddings from scratch (which can take a while depending on the GPU)
    corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

In [ ]:
def search(query):
    # Encode the query using the bi-encoder and find potentially relevant passages
    start_time = time.time()
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query
    end_time = time.time()

    # Output of top-k hits
    print("Input question:", query)
    print("Results (after {:.3f} seconds):".format(end_time - start_time))
    for hit in hits:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']]))

In [ ]:
search(query = "What is the capital of the France?")

In [ ]:
search(query = "What is the best orchestra in the world?")

In [ ]:
search(query = "Number countries Europe")

In [ ]:
search(query = "When did the cold war end?")

In [ ]:
search(query = "How long do cats live?")

In [ ]:
search(query = "How many people live in Toronto?")

In [ ]:
search(query = "Oldest US president")

In [ ]:
search(query = "Coldest place earth")

In [ ]:
search(query = "When was Barack Obama born?")

In [ ]:
search(query = "Paris eiffel tower")

In [ ]:
search(query = "Which US president was killed?")

In [ ]:
search(query="When is Chinese New Year")

In [ ]:
search(query="what is the name of manchester united stadium")

In [ ]:
search(query="who wrote cant get you out of my head lyrics")

In [ ]:
search(query="where does the story the great gatsby take place")

In [ ]:
search(query="who turned out to be the mother on how i met your mother")

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('nq-distilbert-base-v1')

query_embedding = model.encode('How many people live in London?')

#The passages are encoded as [title, text]
passage_embedding = model.encode([['London', 'London has 9,787,426 inhabitants at the 2011 census.']])

print("Similarity:", util.pytorch_cos_sim(query_embedding, passage_embedding))

In [ ]:
query_embedding = model.encode('who turned out to be the mother on how i met your mother')

#The passages are encoded as [title, text]
passage_embedding = model.encode([['The Mother (How I Met Your Mother)', 'The Mother (How I Met Your Mother) Tracy McConnell (colloquial: "The Mother") is the title character from the CBS television sitcom "How I Met Your Mother". The show, narrated by Future Ted (Bob Saget), tells the story of how Ted Mosby (Josh Radnor) met The Mother. Tracy McConnell appears in eight episodes, from "Lucky Penny" to "The Time Travelers", as an unseen character; she was first seen fully in "Something New" and was promoted to a main character in season 9. The Mother is played by Cristin Milioti. The story of how Ted met The Mother is the framing device'],
                                  ['Make It Easy on Me', 'and Pete Waterman on her 1993 album "Good \'N\' Ready", on which a remixed version of the song is included. "Make It Easy On Me", a mid-tempo R&B jam, received good reviews (especially for signalling a different, more soulful and mature sound atypical of the producers\' Europop fare), but failed to make an impact on the charts, barely making the UK top 100 peaking at #99, and peaking at #52 on the "Billboard" R&B charts. The pop group Steps covered the song on their 1999 album "Steptacular". It was sung as a solo by Lisa Scott-Lee. Make It Easy on']])

print("Similarity:", util.pytorch_cos_sim(query_embedding, passage_embedding))

In [ ]:
query_embedding = model.encode('where does the story the great gatsby take place')
passage_embedding = model.encode([['The Great Gatsby',
 'The Great Gatsby The Great Gatsby is a 1925 novel written by American author F. Scott Fitzgerald that follows a cast of characters living in the fictional towns of West Egg and East Egg on prosperous Long Island in the summer of 1922. The story primarily concerns the young and mysterious millionaire Jay Gatsby and his quixotic passion and obsession with the beautiful former debutante Daisy Buchanan. Considered to be Fitzgerald\'s magnum opus, "The Great Gatsby" explores themes of decadence, idealism, resistance to change, social upheaval, and excess, creating a portrait of the Roaring Twenties that has been described as'],
 ['The Producers (1967 film)', '2005 (to coincide with the remake released that year). In 2011, MGM licensed the title to Shout! Factory to release a DVD and Blu-ray combo pack with new HD transfers and bonus materials. StudioCanal (worldwide rights holder to all of the Embassy Pictures library) released several R2 DVD editions and Blu-ray B releases using a transfer slightly different from the North Ameri can DVD and BDs. The Producers (1967 film) The Producers is a 1967 American satirical comedy film written and directed by Mel Brooks and starring Zero Mostel, Gene Wilder, Dick Shawn, and Kenneth Mars. The film was Brooks\'s directorial']
])

print("Similarity:", util.pytorch_cos_sim(query_embedding, passage_embedding))
